In [37]:
import sys
sys.path.append('/mnt/data')  # Adjust if using a different path
from minsearch import Index

In [46]:
from minsearch import Index

In [47]:
!pip install minsearch

In [48]:
import json

In [49]:
with open('documents.json', 'rt') as f_in:
    docs_raw = json.load(f_in)

In [50]:
documents = []

for course_dict in docs_raw:
    for doc in course_dict['documents']:
        doc['course'] = course_dict['course']
        documents.append(doc)

In [51]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

In [53]:
index = Index(
    text_fields=["question", "text", "section"],
    keyword_fields=["course"]
)

SELECT * WHERE course = 'data-engineering-zoomcamp';

In [54]:
q = 'the course has already started, can I still enroll?'

In [57]:
index.fit(documents)

In [59]:
!pip install openai

In [1]:
import openai

client = openai.OpenAI(api_key="")

response = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[
        {"role": "user", "content": "Hello!"}
    ]
)

print(response.choices[0].message.content)



APIConnectionError: Connection error.

In [2]:
client = OpenAI()

NameError: name 'OpenAI' is not defined

In [16]:
response = client.chat.completions.create(
    model='gpt-4o',
    messages=[{"role": "user", "content": q}]
)

response.choices[0].message.content

NameError: name 'client' is not defined

In [17]:
def search(query):
    boost = {'question': 3.0, 'section': 0.5}

    results = index.search(
        query=query,
        filter_dict={'course': 'data-engineering-zoomcamp'},
        boost_dict=boost,
        num_results=5
    )

    return results

In [18]:
def build_prompt(query, search_results):
    prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT: 
{context}
""".strip()

    context = ""
    
    for doc in search_results:
        context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"
    
    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt

In [19]:
def llm(prompt):
    response = client.chat.completions.create(
        model='gpt-4o',
        messages=[{"role": "user", "content": prompt}]
    )
    
    return response.choices[0].message.content

In [20]:
query = 'how do I run kafka?'

def rag(query):
    search_results = search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer

In [21]:
rag(query)

NameError: name 'index' is not defined

In [22]:
rag('the course has already started, can I still enroll?')

NameError: name 'index' is not defined

In [23]:
documents[0]

IndexError: list index out of range

In [24]:
from elasticsearch import Elasticsearch

ModuleNotFoundError: No module named 'elasticsearch'

In [25]:
es_client = Elasticsearch('http://localhost:9200') 

NameError: name 'Elasticsearch' is not defined

In [26]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}

index_name = "course-questions"

es_client.indices.create(index=index_name, body=index_settings)

NameError: name 'es_client' is not defined

In [27]:
documents[0]

IndexError: list index out of range

In [28]:
from tqdm.auto import tqdm

In [29]:
for doc in tqdm(documents):
    es_client.index(index=index_name, document=doc)

0it [00:00, ?it/s]

In [30]:
query = 'I just disovered the course. Can I still join it?'

In [31]:
def elastic_search(query):
    search_query = {
        "size": 5,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^3", "text", "section"],
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "course": "data-engineering-zoomcamp"
                    }
                }
            }
        }
    }

    response = es_client.search(index=index_name, body=search_query)
    
    result_docs = []
    
    for hit in response['hits']['hits']:
        result_docs.append(hit['_source'])
    
    return result_docs

In [32]:
def rag(query):
    search_results = elastic_search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer

In [34]:
rag(query)

NameError: name 'es_client' is not defined